### Importing Data

In [1]:
import numpy as np
import pandas as pd
import json
import gzip # doesn't work for this data
import os
import re
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('stopwords', quiet=True)
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier 
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
exist = os.path.isfile('reddit_submissions.json.gz')
print('Does the file exist in the working directory?', exist)

Does the file exist in the working directory? True


In [3]:
# subreddit multi-class classification
d = pd.read_json('reddit_submissions.json.gz', lines=True, compression='gzip')

### Data Exploration

In [4]:
print('shape of the data', d.shape)
print(d.head(10))

shape of the data (944088, 10)
         author  created_utc     id  num_comments selftext        subreddit  \
0         Crito   1223866465  76r64             0                      ptsd   
1         Crito   1228150531  7goht             0                      ptsd   
2  socialcelebs   1228219003  7guki             0              mentalhealth   
3     [deleted]   1228244460  7gxll             1           EatingDisorders   
4     [deleted]   1228244538  7gxm3             1           EatingDisorders   
5     [deleted]   1228244714  7gxn0             1           EatingDisorders   
6     [deleted]   1228244974  7gxnq             0           EatingDisorders   
7     [deleted]   1228245123  7gxof             0           EatingDisorders   
8     [deleted]   1228245234  7gxot             0           EatingDisorders   
9     [deleted]   1228245285  7gxp4             0           EatingDisorders   

  subreddit_name_prefixed subreddit_type  \
0                  r/ptsd         public   
1          

In [5]:
# frequency of subreddit categories - for now we will leave it as is, but this might change
c = Counter(d['subreddit'])
fewer = [sub for sub in c if c[sub] < 1000]        
print('all subreddit categories and counts', c)
print()
print('subreddit categories with fewer than 1000 examples', fewer)

all subreddit categories and counts Counter({'depression': 489930, 'SuicideWatch': 182855, 'mentalhealth': 53965, 'BPD': 45630, 'socialanxiety': 42566, 'BipolarReddit': 22245, 'schizophrenia': 21549, 'MMFB': 17958, 'alcoholism': 12916, 'ptsd': 11063, 'rapecounseling': 9478, 'dpdr': 7222, 'StopSelfHarm': 6292, 'getting_over_it': 6057, 'Anger': 4772, 'EatingDisorders': 2964, 'survivorsofabuse': 2612, 'feelgood': 1554, 'hardshipmates': 1207, 'psychoticreddit': 549, 'PanicParty': 460, 'traumatoolbox': 244})

subreddit categories with fewer than 1000 examples ['PanicParty', 'psychoticreddit', 'traumatoolbox']


### Initial Model Training

**Preprocessing**

In [6]:
# This is for the 'selftext' column

# count how many elements are in data that we have to drop 
st = d['selftext']
blank = st[st == '']
removed = st[st == '[removed]']
deleted = st[st == '[deleted]']
total = len(blank) + len(removed) + len(deleted)
print('elements to remove from data')
print('blank', len(blank), '|', 'removed', len(removed), '|', 'deleted', len(deleted), '|', 'fraction', total / len(st))

elements to remove from data
blank 67614 | removed 30715 | deleted 151654 | fraction 0.26478781638999754


In [7]:
# deleting selftext data
d = d[~d.selftext.isin(['', '[removed]', '[deleted]'])]
print('new shape of data after selftext deleting', d.shape)

new shape of data after selftext deleting (694105, 10)


In [8]:
# check if our dropping process was successful - we should get 0 for each case
st = d['selftext']
blank = st[st == '']
removed = st[st == '[removed]']
deleted = st[st == '[deleted]']
total = len(blank) + len(removed) + len(deleted)
print('elements to remove from data')
print('blank', len(blank), '|', 'removed', len(removed), '|', 'deleted', len(deleted), '|', 'fraction', total / len(st))
print('Did our data processing work?', total == 0)

elements to remove from data
blank 0 | removed 0 | deleted 0 | fraction 0.0
Did our data processing work? True


We check that the correct rows were deleted from the data. This should be reflected in the reduced sample size from 944088 to 694105. Now we have to also clean the 'num_comments' column. We want to make sure that there are a sufficient number of comments for a story (>=5) so that the row can positively contribute to the model.

In [9]:
# Now we will clean up the 'num_comments' column so that the rows all have >= 5 comments
print('fraction of data that have less than 5 comments', len(d[d.num_comments < 5]) / len(d.num_comments))

fraction of data that have less than 5 comments 0.6589248024434343


In [10]:
# deleting the appropriate num_comments data
d = d[d.num_comments >= 5]
print('new shape of data after deleting num_comments data', d.shape)

new shape of data after deleting num_comments data (236742, 10)


In [11]:
# check if our dropping process was successful - we should get 0 
print('fraction of data that have less than 5 comments after deleting', len(d[d.num_comments < 5]) / len(d.num_comments))

fraction of data that have less than 5 comments after deleting 0.0


Just like before, the sample size has become even smaller. The data went from 694105 to 236742. Our final preprocessing task is to figure out what to do with the subreddit categories that have fewer that 1000 examples. Since it does not make sense to group those categories into other larger ones, we should just leave the subreddit categories the way they are. The main concern with doing this is that it might lead to classifications issues given the small sample size for the 3 subreddit categories mentioned above. 

**Partitioning the data into training, validation, and testing sets (60/20/20 split)**

In [12]:
X = d[['title', 'selftext']]
y = d['subreddit']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5) # random_state is for reproducibility
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=5)

Partitioning the data is rather straight forward. We simply split the data such that 60% is for training, 20% is for validation, and the remaining 20% is for testing. The validation set is used to pick which model we will use. The testing set is only used at the very end to get the out-of-sample performance. To ensure reproducibility, we add a random seed value (5). Now we will get the same results no matter how many times we run this code. 

**Initial Model Training: Multi-Classification**

X variables: 
1. title
2. selftext

Y variables:
1. subreddit

**Models we will use:**
* Naive Bayes
* Stochastic Gradient Descent Classifier
* Random Forest Classifier 

In [13]:
# We need to fit and transform our text data when we vectorize it

# Fit and Transform for Text
class TextSelector(BaseEstimator, TransformerMixin):
    def __init__(self, field):
        self.field = field
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.field]

# Fit and Transform for Numerical
class NumberSelector(BaseEstimator, TransformerMixin):
    def __init__(self, field):
        self.field = field
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[[self.field]]
    
# Tokenize based on Snowball Stemmer (Porter 2)
def Tokenizer(str_input):
    words = re.sub(r"[^A-Za-z0-9\-]", " ", str_input).lower().split()
    snowball_stemmer = nltk.SnowballStemmer('english', ignore_stopwords=True)
    words = [snowball_stemmer.stem(word) for word in words]
    return words

*Naive Bayes*

In [14]:
# Create Naive Bayes Pipeline using FeatureUnion for our 2 predictor variables
nb_classifier = Pipeline([
    ('features', FeatureUnion([
        ('title', Pipeline([('text', TextSelector('title')),
                            ('tfidf', TfidfVectorizer(tokenizer=Tokenizer, stop_words=stopwords.words('english')))]
        )),
        ('selftext', Pipeline([('text', TextSelector('selftext')),
                               ('tfidf', TfidfVectorizer(tokenizer=Tokenizer, stop_words=stopwords.words('english')))]
        ))
    ])),
    ('clf', MultinomialNB())
])

In [15]:
nb_classifier = nb_classifier.fit(X_train, y_train)

In [16]:
nb_predict = nb_classifier.predict(X_val)
nb_success = np.mean(nb_predict == y_val)
print('Naive Bayes Success Rate', nb_success)
print(classification_report(y_val, nb_predict, target_names=list(set(y))))

Naive Bayes Success Rate 0.46722458354233215
                  precision    recall  f1-score   support

 EatingDisorders       0.00      0.00      0.00       187
   hardshipmates       0.90      0.14      0.24      2639
      depression       0.78      0.20      0.32      2223
        feelgood       1.00      0.01      0.01       200
      alcoholism       0.67      0.00      0.00      1063
   BipolarReddit       0.00      0.00      0.00        15
            dpdr       0.00      0.00      0.00       315
 getting_over_it       0.70      0.32      0.44      8802
   socialanxiety       1.00      0.02      0.03       761
            ptsd       0.42      0.94      0.58     14731
           Anger       1.00      0.01      0.03       429
  rapecounseling       0.00      0.00      0.00         2
 psychoticreddit       0.00      0.00      0.00       315
            MMFB       0.00      0.00      0.00        32
    StopSelfHarm       0.00      0.00      0.00      1179
      PanicParty       0.0

*Stochastic Gradient Descent Classification*

In [17]:
# Create Stochastic Gradient Descent Classification Pipeline using FeatureUnion for our 2 predictor variables
sgd_classifier = Pipeline([
    ('features', FeatureUnion([
        ('title', Pipeline([('text', TextSelector('title')),
                            ('tfidf', TfidfVectorizer(tokenizer=Tokenizer, stop_words=stopwords.words('english')))]
        )),
        ('selftext', Pipeline([('text', TextSelector('selftext')),
                               ('tfidf', TfidfVectorizer(tokenizer=Tokenizer, stop_words=stopwords.words('english')))]
        ))
    ])),
    ('clf', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, max_iter=5, random_state=5)) # Ridge Regression
])

In [18]:
sgd_classifier = sgd_classifier.fit(X_train, y_train)

In [19]:
sgd_predict = sgd_classifier.predict(X_val)
sgd_success = np.mean(sgd_predict == y_val)
print('Stochastic Gradient Descent Classification Success Rate', sgd_success)
print(classification_report(y_val, sgd_predict, target_names=list(set(y))))

Stochastic Gradient Descent Classification Success Rate 0.6502811584255128
                  precision    recall  f1-score   support

 EatingDisorders       0.76      0.35      0.48       187
   hardshipmates       0.86      0.54      0.66      2639
      depression       0.77      0.53      0.63      2223
        feelgood       0.85      0.94      0.89       200
      alcoholism       0.91      0.18      0.30      1063
   BipolarReddit       0.60      0.20      0.30        15
            dpdr       0.63      0.22      0.33       315
 getting_over_it       0.60      0.72      0.65      8802
   socialanxiety       0.75      0.86      0.80       761
            ptsd       0.61      0.78      0.69     14731
           Anger       0.90      0.58      0.71       429
  rapecounseling       0.00      0.00      0.00         2
 psychoticreddit       0.00      0.00      0.00       315
            MMFB       0.67      0.06      0.11        32
    StopSelfHarm       0.58      0.05      0.09      1

*Random Forest Classifier*

In [20]:
# Create Naive Bayes Pipeline using FeatureUnion for our 2 predictor variables
rf_classifier = Pipeline([
    ('features', FeatureUnion([
        ('title', Pipeline([('text', TextSelector('title')),
                            ('tfidf', TfidfVectorizer(tokenizer=Tokenizer, stop_words=stopwords.words('english')))]
        )),
        ('selftext', Pipeline([('text', TextSelector('selftext')),
                               ('tfidf', TfidfVectorizer(tokenizer=Tokenizer, stop_words=stopwords.words('english')))]
        ))
    ])),
    ('clf', RandomForestClassifier())
])

In [21]:
rf_classifier = rf_classifier.fit(X_train, y_train)

In [22]:
rf_predict = rf_classifier.predict(X_val)
rf_success = np.mean(rf_predict == y_val)
print('Random Forest Classifier Success Rate', rf_success)
print(classification_report(y_val, rf_predict, target_names=list(set(y))))

Random Forest Classifier Success Rate 0.49634362047572533
                  precision    recall  f1-score   support

 EatingDisorders       0.80      0.04      0.08       187
   hardshipmates       0.56      0.34      0.43      2639
      depression       0.59      0.38      0.46      2223
        feelgood       0.97      0.62      0.76       200
      alcoholism       0.57      0.10      0.16      1063
   BipolarReddit       0.00      0.00      0.00        15
            dpdr       0.28      0.02      0.04       315
 getting_over_it       0.45      0.50      0.47      8802
   socialanxiety       0.83      0.40      0.54       761
            ptsd       0.48      0.74      0.58     14731
           Anger       0.92      0.19      0.31       429
  rapecounseling       0.00      0.00      0.00         2
 psychoticreddit       0.00      0.00      0.00       315
            MMFB       0.00      0.00      0.00        32
    StopSelfHarm       0.39      0.04      0.07      1179
      PanicPa

We train our model by using the FeatureUnion feature of the Pipeline function. This allows us to train our NLP multi-classification model with more than 1 predictor variable. We eliminate stop words because these words typically add no numerical meaning to a sentence. Furthermore, we stem our sentences/words (Snowball Stemming) to add text normalization to our model. This helps our model get to the root of a words without getting confused by complex prefixes or sufixes.

Breaking down the specific models, it appears that the Naive Bayes model predicts very accurately for some of the subreddits, but extremely poorly for others. The other two models are more centered, with predictions ranging around 0.3-0.8.

To look at the accuracy of our models, we simply look at our trained models and apply them to the validation set. We look at the ratio of the amount of correct responses vs the total amount of responses. As you can see above, SGD had the highest validation accuracy = 0.6502811584255128. Therefore, we will be choosing the **SGD model** as our model of choice.

### Model Re-Training

**The only two other columns that may be useful for are 'num_comments' and 'subreddit_type'**

Now we are going to change the subreddit_type column from public=0 and restricted=1

**New Preprocessing**

In [23]:
# Different subreddit types
list(set(d['subreddit_type'].values))

['restricted', 'public', None]

We see that we have to remove all of the rows that have *None* for subreddit type

In [24]:
print('how many None types are there', len(d[d.subreddit_type.isin([None])]))

how many None types are there 216138


Since this value is so large (there would only be ~20k data points left), we are not going to include the subreddit_type variable

**New Model Training**

Partitioning the dataset

In [25]:
X = d[['title', 'selftext', 'num_comments']]
y = d['subreddit']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5) # random_state is for reproducibility
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=5)

X variables:
1. title
2. selftext
3. num_comments

Y variables:
1. subreddit

We are going to use the same 3 models that we used before

*Naive Bayes*

In [26]:
# Create Naive Bayes Pipeline using FeatureUnion for our 4 predictor variables
nb_classifier_mod = Pipeline([
    ('features', FeatureUnion([
        ('title', Pipeline([('text', TextSelector('title')),
                            ('tfidf', TfidfVectorizer(tokenizer=Tokenizer, stop_words=stopwords.words('english')))]
        )),
        ('selftext', Pipeline([('text', TextSelector('selftext')),
                               ('tfidf', TfidfVectorizer(tokenizer=Tokenizer, stop_words=stopwords.words('english')))]
        )),
        ('num_comments', Pipeline([('num', NumberSelector('num_comments'))]
        ))
    ])),
    ('clf', MultinomialNB())
])

In [27]:
nb_classifier_mod = nb_classifier_mod.fit(X_train, y_train)

In [28]:
nb_predict_mod = nb_classifier_mod.predict(X_val)
nb_success_mod = np.mean(nb_predict_mod == y_val)
print('Naive Bayes Modified Success Rate', nb_success_mod)
print(classification_report(y_val, nb_predict_mod, target_names=list(set(y))))

Naive Bayes Modified Success Rate 0.45745663824282584
                  precision    recall  f1-score   support

 EatingDisorders       0.00      0.00      0.00       187
   hardshipmates       0.88      0.10      0.18      2639
      depression       0.76      0.12      0.21      2223
        feelgood       0.00      0.00      0.00       200
      alcoholism       0.00      0.00      0.00      1063
   BipolarReddit       0.00      0.00      0.00        15
            dpdr       0.00      0.00      0.00       315
 getting_over_it       0.63      0.35      0.45      8802
   socialanxiety       1.00      0.00      0.01       761
            ptsd       0.42      0.92      0.58     14731
           Anger       0.00      0.00      0.00       429
  rapecounseling       0.00      0.00      0.00         2
 psychoticreddit       0.00      0.00      0.00       315
            MMFB       0.00      0.00      0.00        32
    StopSelfHarm       0.00      0.00      0.00      1179
      PanicParty 

*Stochastic Gradient Descent Classification*

In [29]:
# Create Stochastic Gradient Descent Classification Pipeline using FeatureUnion for our 2 predictor variables
sgd_classifier_mod = Pipeline([
    ('features', FeatureUnion([
        ('title', Pipeline([('text', TextSelector('title')),
                            ('tfidf', TfidfVectorizer(tokenizer=Tokenizer, stop_words=stopwords.words('english')))]
        )),
        ('selftext', Pipeline([('text', TextSelector('selftext')),
                               ('tfidf', TfidfVectorizer(tokenizer=Tokenizer, stop_words=stopwords.words('english')))]
        )),
        ('num_comments', Pipeline([('num', NumberSelector('num_comments'))]
        ))
    ])),
    ('clf', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, max_iter=5, random_state=5)) # Ridge Regression
])

In [30]:
sgd_classifier_mod = sgd_classifier_mod.fit(X_train, y_train)

In [31]:
sgd_predict_mod = sgd_classifier_mod.predict(X_val)
sgd_success_mod = np.mean(sgd_predict_mod == y_val)
print('Stochastic Gradient Descent Classification Modified Success Rate', sgd_success_mod)
print(classification_report(y_val, sgd_predict_mod, target_names=list(set(y))))

Stochastic Gradient Descent Classification Modified Success Rate 0.4296047942131524
                  precision    recall  f1-score   support

 EatingDisorders       0.79      0.08      0.15       187
   hardshipmates       0.94      0.31      0.46      2639
      depression       0.84      0.21      0.33      2223
        feelgood       0.85      0.45      0.59       200
      alcoholism       0.87      0.07      0.13      1063
   BipolarReddit       0.00      0.00      0.00        15
            dpdr       0.52      0.03      0.07       315
 getting_over_it       0.31      0.97      0.46      8802
   socialanxiety       0.74      0.51      0.61       761
            ptsd       0.81      0.28      0.42     14731
           Anger       0.98      0.18      0.31       429
  rapecounseling       0.00      0.00      0.00         2
 psychoticreddit       1.00      0.00      0.01       315
            MMFB       0.00      0.00      0.00        32
    StopSelfHarm       0.42      0.07      0.

*Random Forest Classifier*

In [32]:
# Create Naive Bayes Pipeline using FeatureUnion for our 2 predictor variables
rf_classifier_mod = Pipeline([
    ('features', FeatureUnion([
        ('title', Pipeline([('text', TextSelector('title')),
                            ('tfidf', TfidfVectorizer(tokenizer=Tokenizer, stop_words=stopwords.words('english')))]
        )),
        ('selftext', Pipeline([('text', TextSelector('selftext')),
                               ('tfidf', TfidfVectorizer(tokenizer=Tokenizer, stop_words=stopwords.words('english')))]
        )),
        ('num_comments', Pipeline([('num', NumberSelector('num_comments'))]
        ))
    ])),
    ('clf', RandomForestClassifier())
])

In [33]:
rf_classifier_mod = rf_classifier_mod.fit(X_train, y_train)

In [34]:
rf_predict_mod = rf_classifier_mod.predict(X_val)
rf_success_mod = np.mean(rf_predict_mod == y_val)
print('Random Forest Classifier Modified Success Rate', rf_success_mod)
print(classification_report(y_val, rf_predict_mod, target_names=list(set(y))))

Random Forest Classifier Modified Success Rate 0.5095435465561393
                  precision    recall  f1-score   support

 EatingDisorders       0.76      0.12      0.20       187
   hardshipmates       0.58      0.39      0.47      2639
      depression       0.60      0.37      0.46      2223
        feelgood       0.92      0.39      0.55       200
      alcoholism       0.57      0.11      0.18      1063
   BipolarReddit       0.00      0.00      0.00        15
            dpdr       0.50      0.06      0.11       315
 getting_over_it       0.45      0.52      0.48      8802
   socialanxiety       0.81      0.44      0.57       761
            ptsd       0.50      0.74      0.59     14731
           Anger       0.97      0.23      0.37       429
  rapecounseling       0.00      0.00      0.00         2
 psychoticreddit       0.14      0.00      0.01       315
            MMFB       0.00      0.00      0.00        32
    StopSelfHarm       0.27      0.02      0.03      1179
     

In [35]:
# comparison between the old and new models
pd.DataFrame([[0.4672, 0.6502, 0.4921], [0.4574, 0.4296, 0.5047]], index=['old', 'new'], columns=['NB', 'SGD', 'RF'])

,NB,SGD,RF
old,0.4672,0.6502,0.4921
new,0.4574,0.4296,0.5047


Looking at the modified models we just created we would choose the Random forest model because it has sthe highest validation score. However, it is also worth noting that the Random forest model is the only model that increased with the addition of the num_comments column. The Stochastic Gradient Descent Model's accuracy decreased significantly with the addition of the num_comments column. Overall, it seems that we are better off only using the two variables and sticking with the original SGD model.

### Out-of-sample accuracy for the original SGD model using the testing set

In [36]:
sgd_predict_final = sgd_classifier.predict(X_test)
sgd_success_final = np.mean(sgd_predict_final == y_test)
print('Stochastic Gradient Descent Classification Final Success Rate', sgd_success_final)
print(classification_report(y_test, sgd_predict_final, target_names=list(set(y))))

Stochastic Gradient Descent Classification Final Success Rate 0.6479756700247101
                  precision    recall  f1-score   support

 EatingDisorders       0.71      0.29      0.41       245
   hardshipmates       0.86      0.54      0.66      3341
      depression       0.78      0.53      0.63      2811
        feelgood       0.83      0.90      0.87       240
      alcoholism       0.92      0.15      0.26      1297
   BipolarReddit       0.20      0.05      0.08        19
            dpdr       0.71      0.21      0.33       400
 getting_over_it       0.61      0.71      0.65     10984
   socialanxiety       0.76      0.88      0.81      1052
            ptsd       0.61      0.78      0.68     18290
           Anger       0.92      0.55      0.69       550
  rapecounseling       0.00      0.00      0.00         4
 psychoticreddit       0.00      0.00      0.00       396
            MMFB       0.75      0.08      0.14        38
    StopSelfHarm       0.50      0.04      0.07 

### Potential Improvements

1. Use more sophisticated models such as neural networks
2. Use a pre-trained model such as word_2_vec
3. Analyze the data in a deeper way
4. Use sentence sentiment relative to other similar sentences
5. Use parameter grid search - I tried this, but it was too computationally expensive and could lead to overfitting

### Conclusion

Most of the explanation can be found above. In the end, the original **SGD** model with only the title and selftext variables is the best.